---
title: Motivation for Non-Self-Intersecting Geometry
subtitle: An Introduction to Simple Closed Curves
date: 2025/01/21
---

# Introduction
Shape optimization is the study of designing shapes that minimize or maximize some quantity of
interest. An example would be designing the wing of an airplane that maximizes the lift-to-drag
ratio. A typical shape optimization process loop consists of three steps:
::::{grid} 1 1 2 3

:::{card}
:header: 1. Shape Representation
Using a parameterization method to represent the shape. The parameters $\phi$ are the design
variables. E.g. splines with their control points as parameters.
:::

:::{card}
:header: 2. Shape Evaluation
Evaluating some characteristic of the shape that is to be minimized or maximized. E.g. lift-to-drag
ratio of a wing.
:::

:::{card}
:header: 3. Shape Improvement
Changing the parameters $\phi$ to design shapes that achieve better characteristics. E.g. gradient
descent to optimize $\phi$.
:::
::::

**We will focus on shape representation.**

In a large class of shape optimization problems the shapes of interest are **simple closed curves**.
Simple closed curves are curves that are *closed*, i.e. they form loops and are *simple*, i.e. they
do not self-intersect. Simple closed curves are also called *Jordan curves*.

Simple closed curves have applications in a diverse set of fields: aerospace design, biomedical
modeling, computer graphics, scientific modelling and simulation, gaming, object recognition etc.

To understand why an optimization problem might be concerned only with simple closed curves consider
the following problem: Imagine that there is a flow going left $\to$ right around the body shown in
[](#boundary_important_inside_spaghetti), and consider the lift-to-drag ratio of this shape. The
flow only sees the boundary of this shape, the complicated spaghetti inside does not have any effect
whatsoever on the properties of the body. All the representation used to describe the inner curves
is wasteful.

:::{figure} assets/boundary_important_inside_spaghetti.svg
:alt: Figure showing a spline curve with a boundary and a complicated mess of self-intersecting spline inside.
:label: boundary_important_inside_spaghetti
:height: 200px
An incoming flow would only interact with the boundary. The complicated internal mess is wasteful
representation.
:::

This is especially crucial when doing optimization. Shape representation methods that can
potentially represent self-intersecting shapes would cause the optimization algorithm to search a
bigger design space than needed. Also, self-intersecting shapes might be physically undesirable or
problematic to deal with in downstream tasks such as shape evaluation. Thus, such a shape
parameterization would require manual tuning during optimization. As an e.g. consider the
optimization happening in [](spline_self_intersecting_during_optimization). We represent the shape
using 12 spline control points which are then fed into a neural network that predicts the shape's
lift-to-drag ratio. During training we used non-self-intersecting shapes as they are the ones of
interest, but during optimization, if after a gradient step the spline starts intersecting the
network struggles to predict its lift-to-drag ratio and steers the optimization in an even worse
direction. This is the classical *distributional shift* problem.
:::{figure} assets/spline_self_intersecting_during_optimization.gif
:label: spline_self_intersecting_during_optimization
:height: 200px
Optimization starts from the dashed red initial airfoil shape which is iteratively modified. We see
that the optimization process suffers from distributional shift. Once a self-intersecting shape is
reached it is iteratively made even worse.
:::

When using shape parameterization that can represent self-intersecting shapes the optimization
algorithm's search has to be made limited, or someone has to sit and manually tune and check for
self-intersection. Sometimes additional losses are added to the objective to prevent
self-intersection. This is a hassle. In effect prevents an automated and aggressive shape space
exploration. In an ideal setting, one would like to leave gradient descent running and go to sleep
and wake up to find the optimal shape. Thus we need a shape parameterization that has the quality of
non-self-intersection in-built and for any set of parameters produces non-self-intersecting shapes
only.

## Naive Usage of Polar Coordinates: The Representation Power Problem.
One simple way of representing non-self-intersecting shapes would be through polar coordinates in
the form:
$$
r = f(\theta), \quad \theta \in [0, 2\pi)
$$
where we would represent $f$ using some parameterized form, e.g. neural networks.

But this representation cannot describe **general** simple closed curves. This is because $r$ is
parameterized as a function of $\theta$ and for a given $\theta$ we can only have one value of $r$.
This means we cannot represent shapes as shown in [](#polar_problem).

:::{figure} assets/polar_problem.svg
:alt: Figure showing a curve that has two r values at a single theta.
:label: polar_problem
:height: 350px
Since we have multiple $r$ values at a single $\theta$ we cannot have a function that can represent
this.
:::

Therefore we would not be able to analyze a fish swimming in water as shown in [](#fish_swimming).
Or if for whatever reason the optimal design was the [Stanford Bunny](#stanford_bunny), we would not
be able to design for it (jokes apart we will use the Stanford Bunny as a target shape to test our
shape representation techniques).

:::::{grid} 2

::::{grid-item}
:::{figure} assets/fish_swimming.svg
:label: fish_swimming
:alt: Fish swimming with its body in a curve shape not representable in polar coordinates.
:height: 350px
A swimming fish with its body curved cannot be represented by a polar parameterization.
:::
::::

::::{grid-item}
:::{figure} assets/stanford_bunny.svg
:label: stanford_bunny
:height: 225px
:alt: The Stanford Bunny shown as a target curve.
The [Stanford Bunny](https://en.wikipedia.org/wiki/Stanford_bunny). We will use it as
a target shape against which we test our parameterization techniques.
:::
::::

:::::

We now discuss a new method for shape parameterization that can describe general simple closed
curves. This method ensures that only non-self-intersecting curves are generated for any combination
of the parameters. The central idea behind this geometry parameterization is the concept of
**_injective networks_** which we discuss next.